## Classwork 3

In this group work, we use the procedure from Lecture 6 slides to decide about the statistical significance of an independent variable. We use a bootstrapping procedure to estimate the reciprocal of a coefficient's $t$-statistic.

1. First, read the advertising data set into Python from the file 'Advertising.csv' on the class Github site. For ease of use, separate the columns into their own Numpy arrays. For example, when reading the CSV, if you assigned the DataFrame to be `advertising`, you can get the 'TV' column as an array by: 

    `tv = advertising['TV'].to_numpy()`

In [240]:
import numpy as np
import pandas as pd

In [242]:
advertising = pd.read_csv("Advertising.csv")
advertising.head()

,Unnamed: 0,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [244]:
tv = advertising['TV'].to_numpy()
radio = advertising['Radio'].to_numpy()
newspaper = advertising['Newspaper'].to_numpy()
sales = advertising['Sales'].to_numpy()

Below, we define a function that will perform multiple linear regression for us. Our assumption is that independent variable values are each in an array, that these arrays are themselves in a list or array.  Also, say that we have a separate array for our response variable ($y$).

In [247]:
# xarrays is a list or array, of arrays that each contain values of an indpt variable
# y is an array of the response variable values
def lin_reg(xarrays, y):
    n = len(xarrays[0])
    A = np.array(list(xarrays) + [np.ones(n)]).T
    AtA = A.T@A
    Aty = A.T@y
    return np.linalg.inv(AtA)@Aty

2. Use the `lin_reg` function above to compute the multiple linear regression coefficients for the advertising data, using the TV, Radio, and Newspaper data as independent variables and using Sales as the response variable. Assign the array of coefficients to `p`.

In [250]:
indpt_variables = [tv, radio, newspaper]
p = lin_reg(indpt_variables, sales)
print(f"Data Coefficients (TV, Radio, Newspaper, Intercept): {p}")

Data Coefficients (TV, Radio, Newspaper, Intercept): [ 4.57646455e-02  1.88530017e-01 -1.03749304e-03  2.93888937e+00]


There are 200 points (observations) in the Advertising data set. To subsample from an array with $n$ points, you need to select a random subset of `[0,1,...,n-1]` (we'll use a Numpy version of `range(n)` which we get with: `np.arange(n)`).

To select a random subset of `[0,1,...,n-1]` of size $m$, you can do either of the following:
> (a) Use the command `np.random.choice(n, size=m, replace=False)`

> (b) Set `my_indices = np.arange(n)` and then randomly _shuffle_ the order of `my_indices` by calling `np.random.shuffle(my_indices)`. (It changes it in place.) Then, slice the first $m$ entries in the shuffled list: `my_indices[:m]`.
*  Here is the documentation for the [Numpy shuffle command](https://numpy.org/doc/stable/reference/random/generated/numpy.random.shuffle.html). Alternatively, you can use the Generator object `np.random.default_rng()`: [documentation here](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.shuffle.html#numpy.random.Generator.shuffle).

In [253]:
# Try out shuffling an array of indices and then taking the first m = 100 of them.
n = 200
m = 100

In [255]:
# 1st Approach: Using np.random.choice
observations_a = np.random.choice(n, size=m, replace=False)
print(observations_a)

[131 162  31 109  99   1 189  14  83 179 130 133  53 166 126 151 118 182
 142 121 185 144   4  77 136  69 128  75  66 172 138  48  65 145 114 143
 191 104 168 161  38  21 101  43 156 194  42  55  17 190 107 180 110 115
 132  59   7   5 176 139  63 120  56  78  44  49 158  45  72  37  52   2
  39 192 111 170 103 152  47 186  20  90 157  13  64  73  74 177  96  92
  51  97   0 113 155  41 127 150 175 147]


In [257]:
# 2nd Approach: Shuffling an array of indices
my_indices = np.arange(n)
np.random.shuffle(my_indices)
observations_b = my_indices[:m]
print(observations_b)

[ 31 124  37  22  11 169  29  40 166 134  91  19  76 189 188 193  68 174
 176  75 198  50 182 162 104  61  51  30 102 115 137 163  79  83 151  81
   3  20  56 119 144 128 108 160  89  14  97 135 191  46 111 172   4  28
 159 105 164 157  24  95 121  86  25  16  15  77  80  74 145  72  67 180
  96 130 116  41 136  93  59 109 129  87 148 120 195  44 173 192 199  63
 196  54  71 140  64   9 118  70 110  52]


3. Either by using the shuffling procedure above, or using `np.random.choice`, get 120 random indices when `n=200`. Now, use this array to get a size 120 random subsample from the Advertising data and compute the linear regression coefficients for that subsample.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Once you have done that, put the procedure into a loop that will iterate the process 100 times. On each iteration, save the regression coefficients found in a variable `subsample_ps`. (At the end, this should contain 100 arrays, each having 4 coefficients.)

In [260]:
n = 200
m = 120
num_iters = 100
subsample_ps = []

In [262]:
for _ in range(num_iters):
    # Picks 120 random indices out of 200
    my_indices = np.random.choice(n, size=m, replace=False)
    
    # Builds the subsampled-x and subsampled-y
    sub_x1 = tv[my_indices]         
    sub_x2 = radio[my_indices]     
    sub_x3 = newspaper[my_indices]  
    sub_y = sales[my_indices]     
    
    # Computes the linear regression coefficients on this subsample
    sub_p = lin_reg([sub_x1, sub_x2, sub_x3], sub_y)
    
    subsample_ps.append(sub_p)

In [264]:
subsample_ps = np.array(subsample_ps)
subsample_ps.shape

(100, 4)

4. In each of the arrays in `subsample_ps`, there is one coefficient for TV, one for Radio, one for Newspaper, and one constant coefficient. Compute the [standard deviation](https://numpy.org/doc/stable/reference/generated/numpy.std.html) of the subsample regression coefficients for the TV variable and then divide it by the TV coefficient from the whole data set, `p[0]` (from number 2 above). Then do the analogous thing for Radio and for Newspaper. 

* Which of the three variables are significant?

In [267]:
# ddof=1 for sample std, source: https://stackoverflow.com/questions/34050491/standard-deviation-in-numpy
tv_std = np.std(subsample_ps[:, 0], ddof=1)        
radio_std = np.std(subsample_ps[:, 1], ddof=1)
newspaper_std = np.std(subsample_ps[:, 2], ddof=1)

In [269]:
tv_ratio = tv_std / p[0]
radio_ratio = radio_std / p[1]
newspaper_ratio = newspaper_std / p[2]

In [271]:
print("TV ratio (std / coeff)        =", tv_ratio)
print("Radio ratio (std / coeff)     =", radio_ratio)
print("Newspaper ratio (std / coeff) =", newspaper_ratio)

TV ratio (std / coeff)        = 0.03621711358426427
Radio ratio (std / coeff)     = 0.04776983028761855
Newspaper ratio (std / coeff) = -4.707846992283929


**TV and Radio are significant while Newspaper is not significant. This means Newspaper is highly inconsistent and contributes much less to predicting Sales.**